In [1]:
import pandas as pd
import os
from geopy.distance import geodesic

# Define the path to your Downloads folder
downloads_path = os.path.join(os.path.expanduser("~"), "Downloads")

# Replace 'your_file.csv' with the name of your CSV file
file_name = 'Ridesharing_S_1.csv'
file_path = os.path.join(downloads_path, file_name)

# Load the CSV file into a DataFrame
df = pd.read_csv(file_path)

driverdf = df[df['Announcement'] < 100000]
riderdf = df[df['Announcement'] > 100000]
riderdf.head()

driverdf = driverdf.sort_values(by='Announcementtime')
riderdf = riderdf.sort_values(by='Announcementtime')
driverdf=driverdf.head(500)
riderdf=riderdf.head(500)

driverdf.head()

FileNotFoundError: [Errno 2] No such file or directory: '/root/Downloads/Ridesharing_S_1.csv'

In [ ]:
#making new columns for variables as given in the research paper
driverdf=driverdf.rename(columns={'Announcementtime':'t','Earliesttime':'e','Latesttime':'l'})
driverdf['q']=driverdf['l']-driverdf['Time_Car-Peak']

riderdf=riderdf.rename(columns={'Announcementtime':'t','Earliesttime':'e','Latesttime':'l'})
riderdf['q']=riderdf['l']-riderdf['Time_Car-Peak']

# create a list of pairing between every announcement

# Add a key column to perform Cartesian product
driverdf['key'] = 1
riderdf['key'] = 1

# Perform the Cartesian product
pairdf = pd.merge(driverdf, riderdf, on='key').drop('key', axis=1)

pairdf['Combined_Announcement'] = pairdf['Announcement_x'].astype(str) + pairdf['Announcement_y'].astype(str)
pairdf = pairdf.drop(columns=['Announcement_x', 'Announcement_y'])

# Get the column names
columns = pairdf.columns.tolist()

# Move the last column to the first position
new_order = [columns[-1]] + columns[:-1]

# Reorder the DataFrame
pairdf = pairdf[new_order]

pairdf.shape
#final

In [ ]:
#pairing preprocessing, removing infeasible pairings.
indices_to_drop = []
for index_row in pairdf.iterrows():
    driverorigin=(row['Origin_Latitude_x'],row['Origin_Longitude_x'])
    driverendpoint=(row['Destination_Latitude_x'],row['Destination_Longitude_x'])
    riderorigin=(row['Origin_Latitude_y'],row['Origin_Longitude_y'])
    riderendpoint=(row['Destination_Latitude_y'],row['Destination_Longitude_y'])
    driverdistance=row['Distance_Car-Peak_x']
    drivertime=row['Time_Car-Peak_x']
    ridertime=row['Time_Car-Peak_y']
    l_x=row['l_x']
    l_y=row['l_y']
    e_x=row['e_x']
    e_y=row['e_y']

k=min(l_y-ridertime-((geodesic(driverorigin, riderorigin).kilometers)*driverdistance/drivertime),l_x-ridertime-((geodesic(riderorigin, driverorigin).kilometers)*driverdistance/drivertime))
if((k-e_x)<0) and ((k+((geodesic(driverorigin, riderorigin).kilometers)*driverdistance/drivertime)-e_y)<0):
    indices_to_drop.append(index)

processdf=pairdf.drop(indices_to_drop)

processdf.shape

In [ ]:
#adding weightage column for net distance savings.
processdf['weight']=0
for index,row in processdf.iterrows():
    driverorigin=(row['Origin_Latitude_x'],row['Origin_Longitude_x'])
    driverendpoint=(row['Destination_Latitude_x'],row['Destination_Longitude_x'])
    riderorigin=(row['Origin_Latitude_y'],row['Origin_Longitude_y'])
    riderendpoint=(row['Destination_Latitude_y'],row['Destination_Longitude_y'])
    drivertriplelength=row['Distance_Car-Peak_x']
    drivertime=row['Time_Car-Peak_x']
    ridertriplelength=row['Distance_Car-Peak_y']
    ridertime=row['Time_Car-Peak_y']
    nomatchlength=drivertriplelength+ridertriplelength
    withmatchlength=geodesic(driverorigin, riderorigin).kilometers+geodesic(riderorigin, riderendpoint).kilometers+geodesic(riderendpoint, driverendpoint).kilometers
    processdf.at[index, 'weight'] = nomatchlength-withmatchlength  # Example operation

weights=processdf['weight'].tolist()

#adding weightage column for distance proximity index
processdf['weight']=0
for index,row in processdf.iterrows():
    drivertriplelength=row['Distance_Car-Peak_x']
    ridertriplelength=row['Distance_Car-Peak_y']
    processdf.at[index, 'weight'] = min(drivertriplelength/ridertriplelength,ridertriplelength/drivertriplelength)  # Example operation

weights=processdf['weight'].tolist()

#adding weightage column for adjusted distance proximity index
processdf['weight']=0
for index,row in processdf.iterrows():
    driverorigin=(row['Origin_Latitude_x'],row['Origin_Longitude_x'])
    driverendpoint=(row['Destination_Latitude_x'],row['Destination_Longitude_x'])
    riderorigin=(row['Origin_Latitude_y'],row['Origin_Longitude_y'])
    riderendpoint=(row['Destination_Latitude_y'],row['Destination_Longitude_y'])
    drivertriplelength=row['Distance_Car-Peak_x']
    drivertime=row['Time_Car-Peak_x']
    ridertriplelength=row['Distance_Car-Peak_y']
    ridertime=row['Time_Car-Peak_y']
    withmatchlength=geodesic(driverorigin, riderorigin).kilometers+geodesic(riderorigin, riderendpoint).kilometers+geodesic(riderendpoint, driverendpoint).kilometers
    processdf.at[index, 'weight'] = (drivertriplelength/withmatchlength)*min(drivertriplelength/ridertriplelength,ridertriplelength/drivertriplelength)

In [ ]:
#normal model no weights
from gurobipy import Model, GRB

drivers=processdf['Announcement_x'].unique()
riders=processdf['Announcement_y'].unique()
possible_matches=set(zip(processdf['Announcement_x'],processdf['Announcement_y']))

model=Model('maximizer')
x=model.addVars(possible_matches,vtype=GRB.BINARY, name="x")
model.setObjective(x.sum(),GRB.MAXIMIZE)

for d in drivers:
    model.addConstr(sum(x[d,r] for r in riders if (d,r) in x)<=1)

for r in riders:
    model.addConstr(sum(x[d,r] for d in drivers if (d,r) in x)<=1)
model.optimize()

In [ ]:
#model with weightage
from gurobipy import Model, GRB

drivers=processdf['Announcement_x'].unique()
riders=processdf['Announcement_y'].unique()
possible_matches=set(zip(processdf['Announcement_x'],processdf['Announcement_y'],processdf['weight']))

model=Model('maximizer')
x=model.addVars([(d,r,weight) for d,r,weight in possible_matches],vtype=GRB.BINARY, name="x")
model.setObjective(sum(x[d,r,weight]*weight for d,r,weight in possible_matches),GRB.MAXIMIZE)

for d in drivers:
    model.addConstr(sum(x[d,r,weight] for driver_match,r,weight in possible_matches if driver_match==d)<1)

for r in riders:
    model.addConstr(sum(x[d,r,weight] for d,rider_match,weight in possible_matches if rider_match==r)<1)
model.optimize()

In [ ]:
#performance evaluation terminal

#matchrate calc
matched2=0
for dr2 in possible_matches:
    if x[dr2].x>0.5:
        matched2=matched2+1
matchrate=matched2*2/(len(drivers)+len(riders))
print(matchrate)

#Average Kilometer Saved (AKS) calc
ktotal=0
matched3=0
for dr in possible_matches:
    if x[dr].x>0.5:
        drivertrip = driverdf[driverdf['Announcement'] == dr[0]]['Distance_Car-Peak']
        drivertriplength=float(drivertrip)
        ridertrip = riderdf[riderdf['Announcement'] == dr[1]]['Distance_Car-Peak']
        ridertriplength=float(ridertrip)
        nomatchlength=drivertriplength+ridertriplength

        driverrorw=driverdf[driverdf['Announcement'] == dr[0]]
        drivrorigin=(float(driverrorw['Origin_Latitude']),float(driverrorw['Origin_Longitude']))
        driverendpoint=(float(driverrorw['Destination_Latitude']),float(driverrorw['Destination_Longitude']))

        riderrerow=riderdf[riderdf['Announcement'] == dr[1]]
        riderorigin=(float(riderrerow['Origin_Latitude']),float(riderrerow['Origin_Longitude']))
        riderendpoint=(float(riderrerow['Destination_Latitude']),float(riderrerow['Destination_Longitude']))

        withmatchlength=geodesic(drivrorigin, riderorigin).kilometers+geodesic(riderorigin, riderendpoint).kilometers+geodesic(riderendpoint, driverendpoint).kilometers
        matched3=matched3+1
        ktotal=ktotal+nomatchlength-withmatchlength
aks=ktotal/matched3
print(aks)